## 浙江省各区域财政收支状况

In [1]:
import polars as pl
import altair as alt
import pandas as pd
import numpy as np

# ==============================================
# 1. 读取市级财政数据
# ==============================================
# 读取收入数据
income_df = pl.read_excel(
    "../data/浙江省市级收支数据.xlsx",
    sheet_name="一般公共预算收入",
    read_options={"header_row": 0}
)

# 读取支出数据
expense_df = pl.read_excel(
    "../data/浙江省市级收支数据.xlsx",
    sheet_name="一般公共预算支出",
    read_options={"header_row": 0}
)

print("市级收入数据列名:", income_df.columns)
print("市级支出数据列名:", expense_df.columns)

# 查看数据
print("\n收入数据预览:")
print(income_df)
print("\n支出数据预览:")
print(expense_df)

市级收入数据列名: ['城市', '2018', '2019', '2020', '2021', '2022', '2023']
市级支出数据列名: ['城市', '2018', '2019', '2020', '2021', '2022', '2023']

收入数据预览:
shape: (11, 7)
┌────────┬──────────┬──────────┬──────────┬──────────┬──────────┬──────────┐
│ 城市   ┆ 2018     ┆ 2019     ┆ 2020     ┆ 2021     ┆ 2022     ┆ 2023     │
│ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      │
│ str    ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      ┆ i64      │
╞════════╪══════════╪══════════╪══════════╪══════════╪══════════╪══════════╡
│ 杭州市 ┆ 18250616 ┆ 19659731 ┆ 20933893 ┆ 23865936 ┆ 24506119 ┆ 26168107 │
│ 宁波市 ┆ 13796865 ┆ 14685072 ┆ 15108432 ┆ 17231388 ┆ 16802309 ┆ 17858565 │
│ 温州市 ┆ 5475765  ┆ 5789706  ┆ 6019813  ┆ 6575549  ┆ 5738500  ┆ 6226842  │
│ 嘉兴市 ┆ 5185536  ┆ 5656945  ┆ 5988000  ┆ 6748000  ┆ 5964742  ┆ 6320216  │
│ 湖州市 ┆ 2870960  ┆ 3160702  ┆ 3365565  ┆ 4135188  ┆ 3872962  ┆ 4104988  │
│ …      ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        ┆ …        │
│ 金华市 ┆ 3926214 

In [3]:
# ==============================================
# 2. 数据清洗和整理
# ==============================================
# 统一列名
years = ["2018", "2019", "2020", "2021", "2022", "2023"]

# 将数据从宽格式转换为长格式（便于分析）
income_long = income_df.melt(
    id_vars=["城市"],
    value_vars=years,
    variable_name="年份",
    value_name="一般公共预算收入"
)

expense_long = expense_df.melt(
    id_vars=["城市"],
    value_vars=years,
    variable_name="年份",
    value_name="一般公共预算支出"
)

# 合并收支数据
city_data = income_long.join(
    expense_long,
    on=["城市", "年份"],
    how="inner"
)

print("\n合并后的市级数据（长格式）:")
print(city_data.head(11))



合并后的市级数据（长格式）:
shape: (11, 4)
┌────────┬──────┬──────────────────┬──────────────────┐
│ 城市   ┆ 年份 ┆ 一般公共预算收入 ┆ 一般公共预算支出 │
│ ---    ┆ ---  ┆ ---              ┆ ---              │
│ str    ┆ str  ┆ i64              ┆ i64              │
╞════════╪══════╪══════════════════╪══════════════════╡
│ 杭州市 ┆ 2018 ┆ 18250616         ┆ 17170834         │
│ 宁波市 ┆ 2018 ┆ 13796865         ┆ 15941000         │
│ 温州市 ┆ 2018 ┆ 5475765          ┆ 8741367          │
│ 嘉兴市 ┆ 2018 ┆ 5185536          ┆ 5888689          │
│ 湖州市 ┆ 2018 ┆ 2870960          ┆ 3975433          │
│ …      ┆ …    ┆ …                ┆ …                │
│ 金华市 ┆ 2018 ┆ 3926214          ┆ 5739988          │
│ 衢州市 ┆ 2018 ┆ 1280960          ┆ 3559423          │
│ 舟山市 ┆ 2018 ┆ 1460243          ┆ 3084897          │
│ 台州市 ┆ 2018 ┆ 4311768          ┆ 6537506          │
│ 丽水市 ┆ 2018 ┆ 1300106          ┆ 4320191          │
└────────┴──────┴──────────────────┴──────────────────┘


C:\Users\13978\AppData\Local\Temp\ipykernel_10412\391720823.py:8: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  income_long = income_df.melt(
C:\Users\13978\AppData\Local\Temp\ipykernel_10412\391720823.py:15: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  expense_long = expense_df.melt(


In [4]:
# ==============================================
# 3. 计算关键指标
# ==============================================
# 计算财政收支差额
city_data = city_data.with_columns([
    (pl.col("一般公共预算收入") - pl.col("一般公共预算支出")).alias("财政收支差额"),
    (pl.col("一般公共预算支出") / pl.col("一般公共预算收入") * 100).alias("支出收入比(%)")
])

# 计算各城市2023年收入占比
total_2023_income = city_data.filter(
    (pl.col("年份") == "2023")
)["一般公共预算收入"].sum()

city_2023_data = city_data.filter(
    (pl.col("年份") == "2023")
).with_columns([
    (pl.col("一般公共预算收入") / total_2023_income * 100).alias("占全省收入比重(%)")
])

# 计算增长情况
# 首先计算每个城市2018-2023年收入增长率
growth_data = []
for city in city_data["城市"].unique():
    city_df = city_data.filter(pl.col("城市") == city).sort("年份")
    
    if len(city_df) > 1:
        income_2018 = city_df.filter(pl.col("年份") == "2018")["一般公共预算收入"][0]
        income_2023 = city_df.filter(pl.col("年份") == "2023")["一般公共预算收入"][0]
        expense_2018 = city_df.filter(pl.col("年份") == "2018")["一般公共预算支出"][0]
        expense_2023 = city_df.filter(pl.col("年份") == "2023")["一般公共预算支出"][0]
        
        income_growth = ((income_2023 - income_2018) / income_2018 * 100)
        expense_growth = ((expense_2023 - expense_2018) / expense_2018 * 100)
        
        # 计算年均复合增长率
        years_count = 5  # 2018-2023共5年
        cagr_income = ((income_2023 / income_2018) ** (1/years_count) - 1) * 100
        cagr_expense = ((expense_2023 / expense_2018) ** (1/years_count) - 1) * 100
        
        growth_data.append({
            "城市": city,
            "收入累计增长(%)": round(income_growth, 2),
            "支出累计增长(%)": round(expense_growth, 2),
            "收入年均复合增长率(%)": round(cagr_income, 2),
            "支出年均复合增长率(%)": round(cagr_expense, 2)
        })

growth_df = pl.DataFrame(growth_data)

print("\n各城市2018-2023年增长情况:")
print(growth_df)


各城市2018-2023年增长情况:
shape: (11, 5)
┌────────┬─────────────────┬─────────────────┬───────────────────────┬───────────────────────┐
│ 城市   ┆ 收入累计增长(%) ┆ 支出累计增长(%) ┆ 收入年均复合增长率(%) ┆ 支出年均复合增长率(%) │
│ ---    ┆ ---             ┆ ---             ┆ ---                   ┆ ---                   │
│ str    ┆ f64             ┆ f64             ┆ f64                   ┆ f64                   │
╞════════╪═════════════════╪═════════════════╪═══════════════════════╪═══════════════════════╡
│ 衢州市 ┆ 59.42           ┆ 70.39           ┆ 9.78                  ┆ 11.25                 │
│ 宁波市 ┆ 29.44           ┆ 40.21           ┆ 5.3                   ┆ 6.99                  │
│ 台州市 ┆ 14.63           ┆ 27.97           ┆ 2.77                  ┆ 5.06                  │
│ 嘉兴市 ┆ 21.88           ┆ 40.44           ┆ 4.04                  ┆ 7.03                  │
│ 绍兴市 ┆ 15.44           ┆ 39.88           ┆ 2.91                  ┆ 6.94                  │
│ …      ┆ …               ┆ …               ┆ …              

In [7]:
# ==============================================
# 4. 区域分组分析
# ==============================================
# 定义区域分组
regions = {
    "杭州市": "核心城市",
    "宁波市": "核心城市",
    "温州市": "浙东沿海",
    "嘉兴市": "浙东沿海", 
    "湖州市": "浙北地区",
    "绍兴市": "浙中地区",
    "金华市": "浙中地区",
    "衢州市": "浙西南",
    "舟山市": "海岛地区",
    "台州市": "浙东沿海",
    "丽水市": "浙西南"
}

# 添加区域列
region_series = []
for city in city_data["城市"]:
    region_series.append(regions.get(city, "其他"))
city_data = city_data.with_columns([
    pl.Series("区域", region_series)
])
print("\n添加区域分组后的市级数据预览:")
print(city_data.head(11))


添加区域分组后的市级数据预览:
shape: (11, 7)
┌────────┬──────┬──────────────────┬──────────────────┬──────────────┬───────────────┬──────────┐
│ 城市   ┆ 年份 ┆ 一般公共预算收入 ┆ 一般公共预算支出 ┆ 财政收支差额 ┆ 支出收入比(%) ┆ 区域     │
│ ---    ┆ ---  ┆ ---              ┆ ---              ┆ ---          ┆ ---           ┆ ---      │
│ str    ┆ str  ┆ i64              ┆ i64              ┆ i64          ┆ f64           ┆ str      │
╞════════╪══════╪══════════════════╪══════════════════╪══════════════╪═══════════════╪══════════╡
│ 杭州市 ┆ 2018 ┆ 18250616         ┆ 17170834         ┆ 1079782      ┆ 94.083586     ┆ 核心城市 │
│ 宁波市 ┆ 2018 ┆ 13796865         ┆ 15941000         ┆ -2144135     ┆ 115.540741    ┆ 核心城市 │
│ 温州市 ┆ 2018 ┆ 5475765          ┆ 8741367          ┆ -3265602     ┆ 159.637366    ┆ 浙东沿海 │
│ 嘉兴市 ┆ 2018 ┆ 5185536          ┆ 5888689          ┆ -703153      ┆ 113.55989     ┆ 浙东沿海 │
│ 湖州市 ┆ 2018 ┆ 2870960          ┆ 3975433          ┆ -1104473     ┆ 138.470512    ┆ 浙北地区 │
│ …      ┆ …    ┆ …                ┆ …                ┆ 

In [9]:
# ==============================================
# 5. 数据分析与输出
# ==============================================
city_2023_data = city_data.filter(
    (pl.col("年份") == "2023")
).with_columns([
    (pl.col("一般公共预算收入") / total_2023_income * 100).alias("占全省收入比重(%)")
])
print("=" * 70)
print("浙江省区域财政发展差异分析（2018-2023年）")
print("=" * 70)

# 2023年各城市收入排名
city_2023_ranked = city_2023_data.sort("一般公共预算收入", descending=True)
print("\n1. 2023年各城市一般公共预算收入排名：")
for i, row in enumerate(city_2023_ranked.to_dicts()):
    print(f"   {i+1:2d}. {row['城市']}: {row['一般公共预算收入']:,.0f}万元 "
          f"({row['占全省收入比重(%)']:.1f}%)")

# 杭州宁波合计占比
hz_nb_income = city_2023_data.filter(
    pl.col("城市").is_in(["杭州市", "宁波市"])
)["一般公共预算收入"].sum()

hz_nb_share = (hz_nb_income / total_2023_income) * 100
print(f"\n2. 杭州-宁波核心引擎：")
print(f"   杭州宁波合计收入: {hz_nb_income:,.0f}万元")
print(f"   占全省收入比重: {hz_nb_share:.1f}%")

# 区域分析
print("\n3. 区域财政收入分布：")
region_summary = city_2023_data.with_columns([
    pl.col("区域")
]).group_by("区域").agg([
    pl.col("一般公共预算收入").sum().alias("区域总收入"),
    pl.col("一般公共预算收入").mean().alias("区域平均收入"),
    pl.col("城市").count().alias("城市数量")
])

region_summary = region_summary.with_columns([
    (pl.col("区域总收入") / total_2023_income * 100).alias("区域占比(%)")
])

print(region_summary.sort("区域总收入", descending=True))

浙江省区域财政发展差异分析（2018-2023年）

1. 2023年各城市一般公共预算收入排名：
    1. 杭州市: 26,168,107万元 (31.7%)
    2. 宁波市: 17,858,565万元 (21.6%)
    3. 嘉兴市: 6,320,216万元 (7.7%)
    4. 温州市: 6,226,842万元 (7.5%)
    5. 绍兴市: 5,787,476万元 (7.0%)
    6. 金华市: 5,257,962万元 (6.4%)
    7. 台州市: 4,942,788万元 (6.0%)
    8. 湖州市: 4,104,988万元 (5.0%)
    9. 衢州市: 2,042,082万元 (2.5%)
   10. 舟山市: 1,935,136万元 (2.3%)
   11. 丽水市: 1,860,556万元 (2.3%)

2. 杭州-宁波核心引擎：
   杭州宁波合计收入: 44,026,672万元
   占全省收入比重: 53.4%

3. 区域财政收入分布：
shape: (6, 5)
┌──────────┬────────────┬──────────────┬──────────┬─────────────┐
│ 区域     ┆ 区域总收入 ┆ 区域平均收入 ┆ 城市数量 ┆ 区域占比(%) │
│ ---      ┆ ---        ┆ ---          ┆ ---      ┆ ---         │
│ str      ┆ i64        ┆ f64          ┆ u32      ┆ f64         │
╞══════════╪════════════╪══════════════╪══════════╪═════════════╡
│ 核心城市 ┆ 44026672   ┆ 2.2013336e7  ┆ 2        ┆ 53.362611   │
│ 浙东沿海 ┆ 17489846   ┆ 5.8299e6     ┆ 3        ┆ 21.198601   │
│ 浙中地区 ┆ 11045438   ┆ 5.522719e6   ┆ 2        ┆ 13.387644   │
│ 浙北地区 ┆ 4104988    ┆ 4

In [10]:
# ==============================================
# 6. 数据可视化
# ==============================================
city_pd = city_data.to_pandas()
city_2023_pd = city_2023_data.to_pandas()
growth_pd = growth_df.to_pandas()

# A. 区域财政收入分布可视化
# 1. 2023年各城市收入柱状图（排序）
chart1 = alt.Chart(city_2023_pd).mark_bar(
    color='#4C78A8',
    opacity=0.8
).encode(
    x=alt.X('一般公共预算收入:Q', 
           axis=alt.Axis(title='一般公共预算收入（万元）', format='~s'),
           sort='descending'),
    y=alt.Y('城市:N', 
           sort=alt.EncodingSortField(field='一般公共预算收入', order='descending'),
           axis=alt.Axis(title='城市')),
    tooltip=['城市', alt.Tooltip('一般公共预算收入:Q', format=',.0f'), 
             alt.Tooltip('占全省收入比重(%)', format='.1f')]
).properties(
    title='浙江省各城市2023年一般公共预算收入排名',
    width=600,
    height=400
)

# 添加数值标签
text1 = chart1.mark_text(
    align='left',
    baseline='middle',
    dx=5,
    fontSize=10
).encode(
    text=alt.Text('一般公共预算收入:Q', format=',.0f')
)

chart1 = chart1 + text1
chart1.show()

alt.LayerChart(...)

In [12]:
# 2. 各城市收入占比饼图
chart2 = alt.Chart(city_2023_pd).mark_arc(
    innerRadius=50
).encode(
    theta=alt.Theta('一般公共预算收入:Q', 
                   stack=True,
                   title='收入金额'),
    color=alt.Color('城市:N',
                   legend=alt.Legend(title="城市", columns=1)),
    tooltip=['城市', alt.Tooltip('一般公共预算收入:Q', format=',.0f'),
             alt.Tooltip('占全省收入比重(%)', format='.1f')]
).properties(
    title='2023年各城市收入占比',
    width=500,
    height=400
)
chart2.show()

alt.Chart(...)

In [13]:
# 3. 各城市收入增长趋势图（2018-2023）
chart3 = alt.Chart(city_pd).mark_line(
    point=True,
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份')),
    y=alt.Y('一般公共预算收入:Q', 
           axis=alt.Axis(title='一般公共预算收入（万元）', format='~s')),
    color=alt.Color('城市:N',
                   legend=alt.Legend(title="城市", columns=1)),
    tooltip=['城市', '年份', alt.Tooltip('一般公共预算收入:Q', format=',.0f')]
).properties(
    title='浙江省各城市一般公共预算收入增长趋势（2018-2023）',
    width=700,
    height=500
)
chart3.show()

alt.Chart(...)

In [14]:
# 4. 区域收入对比箱线图
chart4 = alt.Chart(city_pd).mark_boxplot(
    extent='min-max'
).encode(
    x=alt.X('区域:N', 
           axis=alt.Axis(title='区域')),
    y=alt.Y('一般公共预算收入:Q',
           axis=alt.Axis(title='一般公共预算收入（万元）', format='~s'),
           scale=alt.Scale(zero=False)),
    color=alt.Color('区域:N', legend=None)
).properties(
    title='各区域财政收入分布箱线图',
    width=600,
    height=400
)
chart4.show()

alt.Chart(...)

In [20]:
# 5. 收入-支出散点图（2023年）
chart5 = alt.Chart(city_2023_pd).mark_circle(
    size=200,
    opacity=0.7
).encode(
    x=alt.X('一般公共预算收入:Q',
           axis=alt.Axis(title='一般公共预算收入（万元）', format='~s')),
    y=alt.Y('一般公共预算支出:Q',
           axis=alt.Axis(title='一般公共预算支出（万元）', format='~s')),
    color=alt.Color('城市:N',
                   legend=alt.Legend(title="城市")),
    size=alt.Size('一般公共预算收入:Q', legend=None),
    tooltip=['城市', 
             alt.Tooltip('一般公共预算收入:Q', format=',.0f'),
             alt.Tooltip('一般公共预算支出:Q', format=',.0f')]
).properties(
    title='2023年各城市财政收入与支出关系',
    width=600,
    height=400
)

# 添加趋势线
trend_line = chart5.transform_regression(
    '一般公共预算收入', '一般公共预算支出'
).mark_line(
    color='gray',
    strokeDash=[5, 5]
)

chart5 = chart5 + trend_line
chart5.show()

alt.LayerChart(...)

In [16]:
# 6. 增长率对比柱状图
growth_melt = growth_pd.melt(
    id_vars=['城市'],
    value_vars=['收入累计增长(%)', '支出累计增长(%)'],
    var_name='指标',
    value_name='增长率(%)'
)

chart6 = alt.Chart(growth_melt).mark_bar(
    opacity=0.8
).encode(
    x=alt.X('城市:N', axis=alt.Axis(title='城市'), sort='-y'),
    y=alt.Y('增长率(%):Q', axis=alt.Axis(title='增长率(%)')),
    xOffset=alt.XOffset('指标:N'),
    color=alt.Color('指标:N',
                   scale=alt.Scale(
                       domain=['收入累计增长(%)', '支出累计增长(%)'],
                       range=['#4C78A8', '#F58518']
                   ),
                   legend=alt.Legend(title="指标类型"))
).properties(
    title='各城市2018-2023年收入与支出累计增长率对比',
    width=700,
    height=400
)
chart6.show()

alt.Chart(...)

In [17]:

# 7. 地理分布热力图（模拟）
# 创建模拟的地理坐标（实际应用时应使用真实坐标）
cities_coords = {
    "杭州市": [120.15, 30.28],
    "宁波市": [121.55, 29.88],
    "温州市": [120.70, 28.00],
    "嘉兴市": [120.76, 30.77],
    "湖州市": [120.09, 30.90],
    "绍兴市": [120.58, 30.01],
    "金华市": [119.65, 29.08],
    "衢州市": [118.88, 28.97],
    "舟山市": [122.20, 30.00],
    "台州市": [121.43, 28.66],
    "丽水市": [119.92, 28.45]
}

# 创建地理数据
geo_data = []
for city, coords in cities_coords.items():
    city_row = city_2023_pd[city_2023_pd['城市'] == city]
    if len(city_row) > 0:
        income = city_row['一般公共预算收入'].values[0]
        geo_data.append({
            '城市': city,
            '经度': coords[0],
            '纬度': coords[1],
            '一般公共预算收入': income,
            '收入等级': np.log10(income)  # 使用对数尺度便于显示
        })

geo_df = pd.DataFrame(geo_data)

chart7 = alt.Chart(geo_df).mark_circle(
    size=300,
    opacity=0.7
).encode(
    longitude='经度:Q',
    latitude='纬度:Q',
    size=alt.Size('一般公共预算收入:Q', 
                 legend=alt.Legend(title="收入规模"),
                 scale=alt.Scale(range=[50, 500])),
    color=alt.Color('收入等级:Q',
                   scale=alt.Scale(scheme='viridis'),
                   legend=alt.Legend(title="收入等级")),
    tooltip=['城市', alt.Tooltip('一般公共预算收入:Q', format=',.0f')]
).properties(
    title='浙江省各城市财政收入地理分布（2023年）',
    width=600,
    height=500
)
chart7.show()


alt.Chart(...)